In [1]:
"""
Mask R-CNN
Configurations and data loading code for MS COCO.

Copyright (c) 2017 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla

------------------------------------------------------------

Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:

    # Train a new model starting from pre-trained COCO weights
    python3 coco.py train --dataset=/path/to/coco/ --model=coco

    # Train a new model starting from ImageNet weights
    python3 coco.py train --dataset=/path/to/coco/ --model=imagenet

    # Continue training a model that you had trained earlier
    python3 coco.py train --dataset=/path/to/coco/ --model=/path/to/weights.h5

    # Continue training the last model you trained
    python3 coco.py train --dataset=/path/to/coco/ --model=last

    # Run COCO evaluatoin on the last model you trained
    python3 coco.py evaluate --dataset=/path/to/coco/ --model=last
"""
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import numpy as np

# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
sys.path.append('../')
import pprint as pp
pp = pp.PrettyPrinter(indent=4)
import argparse
from   mrcnn.coco import CocoDataset, CocoConfig, evaluate_coco, build_coco_results
from   mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.dataset as dataset

# Root directory of the project
ROOT_DIR = os.getcwd()
ROOT_DIR = 'E:\Models'
# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
COCO_DATASET_PATH = 'E:\MLDatasets\coco2014'
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "mrcnn_coco_logs")

print(COCO_MODEL_PATH)

 breakpoint 1
 breakpoint 2


D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


 Initialize CocoConfig object - super
E:\Models\mask_rcnn_coco.h5


In [2]:
import tensorflow as tf
import keras.backend as KB 
# kears.backend.tensorflow_backend import set_session


##------------------------------------------------------------------------------------
## setup tf session and debugging 
##------------------------------------------------------------------------------------
# keras_backend.set_session(tf_debug.LocalCLIDebugWrapperSession(tf.Session()))

#     if 'tensorflow' == KB.backend():
#         from tensorflow.python import debug as tf_debug
#         config = tf.ConfigProto(
#                 device_count = {'GPU': 0}
#             )
#         tf_sess = tf.Session(config=config)    
#         tf_sess = tf_debug.LocalCLIDebugWrapperSession(tf_sess)
#         KB.set_session(tf_sess)


##------------------------------------------------------------------------------------
## force no GPU usage
##------------------------------------------------------------------------------------

# if 'tensorflow' == KB.backend():
#     config = tf.ConfigProto(
#             device_count = {'GPU': 0}
#         )
#     tf_sess = tf.Session(config=config)    
#     KB.set_session(tf_sess)

##------------------------------------------------------------------------------------
## limit GPU usage
##------------------------------------------------------------------------------------

#   tfconfig = tf.ConfigProto(
#               gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5),
#               device_count = {'GPU': 1}
#              )    
#     tfconfig = tf.ConfigProto()
#     tfconfig.gpu_options.allow_growth=True
#     tfconfig.gpu_options.visible_device_list = "0"
#     tfconfig.gpu_options.per_process_gpu_memory_fraction=0.5
#     tf_sess = tf.Session(config=tfconfig)
#     set_session(tf_sess)
##------------------------------------------------------------------------------------

# tf_config = tf.ConfigProto()
# tf_config.gpu_options.per_process_gpu_memory_fraction = 0.55
# set_session(tf.Session(config=tf_config))

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.55)
# set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))

In [3]:

# Parse command line arguments
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
parser.add_argument("command",
                    metavar="<command>",
                    help="'train' or 'evaluate' on MS COCO")
parser.add_argument('--dataset', required=False,
                    default=COCO_DATASET_PATH,                    
                    metavar="/path/to/coco/",
                    help='Directory of the MS-COCO dataset')
parser.add_argument('--model', required=False,
                    default=COCO_MODEL_PATH,                    
                    metavar="/path/to/weights.h5",
                    help="Path to weights .h5 file or 'coco', 'last','imagenet' ")
parser.add_argument('--logs', required=False,
                    default=DEFAULT_LOGS_DIR,
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')
parser.add_argument('--limit', required=False,
                    default=500,
                    metavar="<image count>",
                    help='Images to use for evaluation (defaults=500)')
# args = parser.parse_args()
# args = parser.parse_args("train --dataset E:\MLDatasets\coco2014 --model mask_rcnn_coco.h5 --limit 10".split())
args = parser.parse_args("train  --limit 10".split())
# pp.pprint(options)
pp.pprint(args)
print(" Model     (COCO_MODEL_PATH)  : ", args.model, '  ', COCO_MODEL_PATH)
print(" Dataset   (COCO_DATASET_PATH): ", args.dataset, '  ',COCO_DATASET_PATH)
print(" Ckpt/Logs (DEFAULT_LOGS_DIR) : ", args.logs, '  ', DEFAULT_LOGS_DIR)
print(" Limit:   ", args.limit)




Namespace(command='train', dataset='E:\\MLDatasets\\coco2014', limit='10', logs='E:\\Models\\mrcnn_coco_logs', model='E:\\Models\\mask_rcnn_coco.h5')
 Model     (COCO_MODEL_PATH)  :  E:\Models\mask_rcnn_coco.h5    E:\Models\mask_rcnn_coco.h5
 Dataset   (COCO_DATASET_PATH):  E:\MLDatasets\coco2014    E:\MLDatasets\coco2014
 Ckpt/Logs (DEFAULT_LOGS_DIR) :  E:\Models\mrcnn_coco_logs    E:\Models\mrcnn_coco_logs
 Limit:    10


In [4]:
# Configurations
if args.command == "train":
    config = CocoConfig()
else:
    class InferenceConfig(CocoConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
        DETECTION_MIN_CONFIDENCE = 0
    config = InferenceConfig()
config.display()

 Initialize config object - super

Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
EPOCHS_TO_RUN                  0
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LAST_EPOCH_RAN                 0
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE            

In [5]:
# Create model
if args.command == "train":
    print('setup model for Training ')
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)
else:
    print('setup model for Inference ')
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=args.logs)

setup model for Training 
>>> Set_log_dir() -- model dir is  E:\Models\mrcnn_coco_logs
    model_path           :    None
    config.LAST_EPOCH_RAN:    0
    set_log_dir: Checkpoint path set to : E:\Models\mrcnn_coco_logs\coco20180502T1930\mask_rcnn_coco_{epoch:04d}.h5
    self.epoch set to 0 

>>> Resnet Graph 
     Input_image shape : (?, 1024, 1024, 3)
     After ZeroPadding2D  : (?, 1030, 1030, 3) (?, 1030, 1030, 3)
     After Conv2D padding : (?, 512, 512, 64) (?, 512, 512, 64)
     After BatchNorm      : (?, 512, 512, 64) (?, 512, 512, 64)
     After MaxPooling2D   : (?, 256, 256, 64) (?, 256, 256, 64)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 256, 256, 256)
     FPN P3 shape : (None, 128, 128, 256)
     FPN P4 shape : (None, 64, 64, 256)
     FPN P5 shape : (None, 32, 32, 256)
     FPN P6 shape : (None, 16, 16, 256)

>>> RPN Layer 
     Input_feature_map shape : (?, ?, ?, 256)
     anchors_per_location    : 3
     depth                   : 256
     Inp

   FCN fully connected 2 (fcn_fc2) shape is :  (2, 128, 128, 2048)
   FCN final conv2d (fcn_classify) shape is :  (None, 128, 128, 81)
   h_factor :  8.0 w_factor :  8.0

>>> BilinearUpSampling2D layer
     data_format :  channels_last
     size        :  (8.0, 8.0)
     target_size :  None
     input_spec  :  [InputSpec(ndim=4)]
     call resize_images_bilinear with size:  (8.0, 8.0)
     CHANNELS LAST: X:  (2, 128, 128, 81)  KB.int_shape() :  (None, 128, 128, 81)
     target_height   :  None  target_width  :  None
     new_shape (2):  (2,) (2,)
     new_shape (3):  (2,) (2,)
     X after image.resize_bilinear:  (2, ?, ?, 81)
     Dimensions of X after set_shape() :  (2, 1024, 1024, 81)
    BilinearUpSampling2D. compute_output_shape()
   FCN output (fcn_bilinear) shape is :  (2, 1024, 1024, 81) Keras tensor  True
   fcn_output  shape is :  (None, 1024, 1024, 81)  Keras tensor  True

>>> rpn_bbox_loss_graph
    rpn_match size : (?, ?)
    rpn_bbox  size : (?, ?, 4)
    tf default sessi

In [6]:
# Select weights file to load
#-----------------------------------------------------
print(' args.model: ', args.model)
# print(' model.find_last() :',model.find_last())
if args.model.lower() == "coco":
    model_path = COCO_MODEL_PATH
elif args.model.lower() == "last":
    # Find last trained weights
    model_path = model.find_last()[1]
elif args.model.lower() == "imagenet":
    # Start from ImageNet trained weights
    model_path = model.get_imagenet_weights()
else:
    model_path = args.model

    print(' model path ', model_path)

 args.model:  E:\Models\mask_rcnn_coco.h5
 model path  E:\Models\mask_rcnn_coco.h5


In [7]:
# Load weights
#-----------------------------------------------------
print("Loading weights ", model_path)
model.load_weights(model_path, by_name=True)


Loading weights  E:\Models\mask_rcnn_coco.h5
>>> load_weights()
    load_weights: Loading weights from: E:\Models\mask_rcnn_coco.h5
    load_weights: Log directory set to : E:\Models\mask_rcnn_coco.h5
>>> Set_log_dir() -- model dir is  E:\Models\mrcnn_coco_logs
    model_path           :    E:\Models\mask_rcnn_coco.h5
    config.LAST_EPOCH_RAN:    0
    set_log_dir: model_path (input) is : E:/Models/mask_rcnn_coco.h5  
    set_log_dir: self.epoch set to 0  (Next epoch to run)
    set_log_dir: tensorboard path: E:\Models\mrcnn_coco_logs\tensorboard
    set_log_dir: Checkpoint path set to : E:\Models\mrcnn_coco_logs\coco20180502T1930\mask_rcnn_coco_{epoch:04d}.h5
    self.epoch set to 0 
    Load weights complete :  E:\Models\mask_rcnn_coco.h5


'E:\\Models\\mask_rcnn_coco.h5'

### Setup Training and Validation datasets. 
#### Use the training set and 35K from the  validation set, as as in the Mask RCNN paper.

In [8]:
# Training dataset. Use the training set and 35K from the
# validation set, as as in the Mask RCNN paper.
dataset_train = CocoDataset()
dataset_train.load_coco(args.dataset, "train")
dataset_train.load_coco(args.dataset, "val35k")
dataset_train.prepare()

# Validation dataset
dataset_val = CocoDataset()
dataset_val.load_coco(args.dataset, "minival")
dataset_val.prepare()


loading annotations into memory...
Done (t=16.19s)
creating index...
index created!
loading annotations into memory...
Done (t=8.35s)
creating index...
index created!
loading annotations into memory...
Done (t=1.29s)
creating index...
index created!


In [9]:
print('\n Outputs: ') 
pp.pprint(model.keras_model.outputs)

trainable = model.get_trainable_layers()
for i in trainable:
    print(' Layer:', i.name)


 Outputs: 
[   <tf.Tensor 'output_rois/mul:0' shape=(2, ?, ?) dtype=float32>,
    <tf.Tensor 'proposal_targets/target_class_ids:0' shape=(2, ?) dtype=int32>,
    <tf.Tensor 'proposal_targets/target_bbox:0' shape=(2, ?, ?) dtype=float32>,
    <tf.Tensor 'proposal_targets/target_mask:0' shape=(2, ?, ?, ?) dtype=float32>,
    <tf.Tensor 'rpn_class_logits/concat:0' shape=(?, ?, 2) dtype=float32>,
    <tf.Tensor 'proposal_rois/packed_2:0' shape=(2, ?, ?) dtype=float32>,
    <tf.Tensor 'rpn_class/concat:0' shape=(?, ?, 2) dtype=float32>,
    <tf.Tensor 'rpn_bbox/concat:0' shape=(?, ?, 4) dtype=float32>,
    <tf.Tensor 'mrcnn_class_logits/Reshape_1:0' shape=(?, 200, 81) dtype=float32>,
    <tf.Tensor 'mrcnn_class/Reshape_1:0' shape=(?, 200, 81) dtype=float32>,
    <tf.Tensor 'mrcnn_bbox/Reshape:0' shape=(?, 200, 81, 4) dtype=float32>,
    <tf.Tensor 'mrcnn_mask/Reshape_1:0' shape=(?, 200, 28, 28, 81) dtype=float32>,
    <tf.Tensor 'fcn_bilinear/ResizeBilinear:0' shape=(2, 1024, 1024, 81) dty

###  Training - Stage 1

In [ ]:
# Training - Stage 1
model.config.STEPS_PER_EPOCH = 2
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=2,
            layers='fcn')


Training network heads

Selecting layers to train
-------------------------
Layer    Layer Name               Layer Type
In model:  rpn_model
212  fcn_block1_conv1       (Conv2D)
214  fcn_block1_conv2       (Conv2D)
218  fcn_block2_conv1       (Conv2D)
220  fcn_block2_conv2       (Conv2D)
224  fcn_block3_conv1       (Conv2D)
226  fcn_block3_conv2       (Conv2D)
228  fcn_block3_conv3       (Conv2D)
232  fcn_fc1                (Conv2D)
236  fcn_fc2                (Conv2D)
240  fcn_classify           (Conv2D)
   keras model add loss for  Tensor("fcn_loss/Reshape_2:0", shape=(1, 1), dtype=float32)
   keras model add loss for  Tensor("fcn_norm_loss/Reshape_6:0", shape=(1, 1), dtype=float32)
Starting at epoch 0 of 2 epochs. LR=0.001

Steps per epochs 2 
Checkpoint Path: E:\Models\mrcnn_coco_logs\coco20180502T1930\mask_rcnn_coco_{epoch:04d}.h5
Epoch 1/2


D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\scipy\ndimage\interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


###  Training - Stage 2 - Finetune layers from ResNet stage 4 and up

In [ ]:
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=120,
            layers='4+')



### Training - Stage 3 - Fine tune all layers

In [ ]:
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=160,
            layers='all')

In [ ]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
layers    = [layer for layer in mdl.layers]          # all layer outputs
len(layers)

In [ ]:
for i in range(len(layers)):
    lay = layers[i]
    print('** Layer {} , ({})'.format(i, lay.name))
    print('===========================================')
    for innode in lay._inbound_nodes:
        print('\t Inbound Node:  ', innode)
        print('\t               From Layers: ', [inlayer.name for inlayer in innode.inbound_layers])
        print('\t                    Shapes: ', [inshape for inshape in innode.input_shapes])
        print('\t               To   Layers: ', innode.outbound_layer.name)
#         pp.pprint(innode.__dict__)
#         print('\t                    Shapes: ', [outshape for outshape in innode.output_shapes])        
    print('\t-------------------------------------------')
    print('\t Layer:', lay.name, '\t\t type  ' , lay)
#     print('-------------------------------------------')
#     pp.pprint(lay.__dict__)
#     print('-------------------------------------------')
    print('\t     Layer Output Shapes: ', [outshape for outshape in innode.output_shapes])        
    print('\t-------------------------------------------')
    for outnode in lay._outbound_nodes:
        print('\t Outbound Node: ', outnode)
#         print('\t               From Layers: ', [inlayer.name for inlayer in outnode.inbound_layers])
#         print('\t                    Shapes: ', [inshape for inshape in outnode.input_shapes])
        print('\t               To   Layer: ', outnode.outbound_layer.name)
#         print('                    Shapes: ', [outshape for outshape in outnode.output_shapes])    
    
#         pp.pprint(outnode.__dict__)
    print('===========================================')

In [ ]:
model.keras_model.summary()

In [ ]:
def generate_anchors(scales, ratios, shape, feature_stride, anchor_stride):
    """
    scales: 1D array of anchor sizes in pixels. Example: [32, 64, 128]
    ratios: 1D array of anchor ratios of width/height. Example: [0.5, 1, 2]
    shape: [height, width] spatial shape of the feature map over which
            to generate anchors.
    feature_stride: Stride of the feature map relative to the image in pixels.
    anchor_stride: Stride of anchors on the feature map. For example, if the
        value is 2 then generate anchors for every other feature map pixel.
    """
    print('Generate Anchors: \n ================== \n scales: ',scales, 'Anchor Ratios:', ratios, 'Anchor stride', anchor_stride,'Feat. Shape:', shape, ' Feat stride', feature_stride)
    # Get all combinations of scales and ratios
    scales, ratios = np.meshgrid(np.array(scales), np.array(ratios))
    scales = scales.flatten()
    ratios = ratios.flatten()
    print('Scales: ',scales,'\n Ratios:', ratios)
    # Enumerate heights and widths from scales and ratios
    heights = scales / np.sqrt(ratios)
    widths = scales * np.sqrt(ratios)

    print(' heights: ',heights,'\n widths: ', widths)
    
    # Enumerate shifts in feature space
    shifts_y = np.arange(0, shape[0], anchor_stride) * feature_stride
    shifts_x = np.arange(0, shape[1], anchor_stride) * feature_stride
#     print(' shifts_x : ',shifts_x.shape, '\n shifts_y : ',  shifts_y.shape )    
    shifts_x, shifts_y = np.meshgrid(shifts_x, shifts_y)
#     print(' shifts_x : ',shifts_x.shape,shifts_x,'\n shifts_y : ',  shifts_y.shape,shifts_y)
    
    # Enumerate combinations of shifts, widths, and heights
    box_widths, box_centers_x = np.meshgrid(widths, shifts_x)
    box_heights, box_centers_y = np.meshgrid(heights, shifts_y)
#     print('box_widths : ',box_widths.shape,'\n',box_widths,'\n box_centers_x: ',  box_centers_x.shape,'\n',box_centers_x)
#     print('box_heights : ',box_heights.shape,'\n',box_heights,'\n box_centers_y: ',  box_centers_y.shape,'\n',box_centers_y)

    # Reshape to get a list of (y, x) and a list of (h, w)
    # This will create a array of (y,x) which has 
    box_centers = np.stack([box_centers_y, box_centers_x], axis=2)
#     print('box_centers : ', box_centers.shape,'\n',box_centers)
    box_centers = box_centers.reshape([-1, 2])
    box_sizes   = np.stack([box_heights, box_widths], axis=2).reshape([-1, 2])
#     print('box_centers : ', box_centers.shape,'\n',box_centers)
#     print('box_sizes   : ', box_sizes.shape,'\n',box_sizes)

    # Convert to corner coordinates (y1, x1, y2, x2)
    boxes = np.concatenate([box_centers - 0.5 * box_sizes, box_centers + 0.5 * box_sizes], axis=1)
#     print('boxes   : ', boxes.shape,'\n',boxes)

    return boxes

In [ ]:
# self.anchors = utils.generate_pyramid_anchors(config.RPN_ANCHOR_SCALES,
#                                               config.RPN_ANCHOR_RATIOS,
#                                               config.BACKBONE_SHAPES,
#                                               config.BACKBONE_STRIDES,
#                                               config.RPN_ANCHOR_STRIDE)
# Anchors
# [anchor_count, (y1, x1, y2, x2)]
scales = config.RPN_ANCHOR_SCALES
ratios = config.RPN_ANCHOR_RATIOS
anchor_stride   = config.RPN_ANCHOR_STRIDE
feature_shapes  = config.BACKBONE_SHAPES
feature_strides = config.BACKBONE_STRIDES

print ('Anchoo scales: ',scales, 'Anchor Ratios:', ratios, 'Anchor stride', anchor_stride)
print(feature_shapes, '\n Feature strides', feature_strides)
anchors = []
for i in range(len(scales)):    
    anchors.append(generate_anchors(scales[i], ratios, feature_shapes[i],
                                    feature_strides[i], anchor_stride))
    
# return np.concatenate(anchors, axis=0)

In [ ]:
for i in anchors :
    print(i.shape)
np.concatenate(anchors, axis=0).shape